In [ ]:
module type Monad = sig
  type 'a t

  val return : 'a -> 'a t

  val bind : 'a t -> ('a -> 'b t) -> 'b t

  (* infix operator for bind *)
  val ( >>= ) : 'a t -> ('a -> 'b t) -> 'b t

  (* infix operator for bind *)
  val ( let* ) : 'a t -> ('a -> 'b t) -> 'b t
end

In [ ]:
module OptionMonad : Monad with type 'a t = 'a option = struct
  type 'a t = 'a option

  let return x = Some x

  let bind o f = match o with None -> None | Some x -> f x

  let ( >>= ) = bind

  let ( let* ) = bind
end

In [ ]:
(*
    Law of Functor
    1. fmap id = id
    2. fmap (f . g) = fmap f . fmap g
*)

module type Functor = sig
  type 'a t

  val fmap : ('a -> 'b) -> 'a t -> 'b t

  (* infix operator for fmap *)
  val ( <!> ) : ('a -> 'b) -> 'a t -> 'b t
  
  (* infix operator for fmap *)
  val ( let+ ) : 'a t -> ('a -> 'b) -> 'b t
end

In [118]:
module OptionFunctor : Functor with type 'a t = 'a option = struct
  type 'a t = 'a option

  let fmap f = function None -> None | Some x -> Some (f x)

  let ( <!> ) = fmap

  let ( let+ ) x f = fmap f x
end

module OptionFunctor :
  sig
    type 'a t = 'a option
    val fmap : ('a -> 'b) -> 'a t -> 'b t
    val ( <!> ) : ('a -> 'b) -> 'a t -> 'b t
    val ( let+ ) : 'a t -> ('a -> 'b) -> 'b t
  end


In [119]:
module type Applicative = sig
  type 'a t

  val apply : ('a -> 'b) t -> 'a t -> 'b t

  val product : 'a t -> 'b t -> ('a * 'b) t

  (* infix operator for apply *)
  val ( <*> ) : ('a -> 'b) t -> 'a t -> 'b t

  (* infix operator for product *)
  val ( and+ ) : 'a t -> 'b t -> ('a * 'b) t

  val lift2 : ('a -> 'b -> 'c) -> 'a t -> 'b t -> 'c t

  val lift3 : ('a -> 'b -> 'c -> 'd) -> 'a t -> 'b t -> 'c t -> 'd t
end

module type Applicative =
  sig
    type 'a t
    val apply : ('a -> 'b) t -> 'a t -> 'b t
    val product : 'a t -> 'b t -> ('a * 'b) t
    val ( <*> ) : ('a -> 'b) t -> 'a t -> 'b t
    val ( and+ ) : 'a t -> 'b t -> ('a * 'b) t
    val lift2 : ('a -> 'b -> 'c) -> 'a t -> 'b t -> 'c t
    val lift3 : ('a -> 'b -> 'c -> 'd) -> 'a t -> 'b t -> 'c t -> 'd t
  end


In [120]:
module OptionApplicative : Applicative with type 'a t = 'a option =
struct
  include OptionFunctor

  type 'a t = 'a option

  let apply f o = match (f, o) with Some fn, Some x -> Some (fn x) | _ -> None

  let product o1 o2 =
    match (o1, o2) with Some x, Some y -> Some (x, y) | _ -> None

  let ( <*> ) = apply

  let ( and+ ) = product

  let lift2 f o1 o2 = f <!> o1 <*> o2

  let lift3 f o1 o2 o3 = f <!> o1 <*> o2 <*> o3
end

module OptionApplicative :
  sig
    type 'a t = 'a option
    val apply : ('a -> 'b) t -> 'a t -> 'b t
    val product : 'a t -> 'b t -> ('a * 'b) t
    val ( <*> ) : ('a -> 'b) t -> 'a t -> 'b t
    val ( and+ ) : 'a t -> 'b t -> ('a * 'b) t
    val lift2 : ('a -> 'b -> 'c) -> 'a t -> 'b t -> 'c t
    val lift3 : ('a -> 'b -> 'c -> 'd) -> 'a t -> 'b t -> 'c t -> 'd t
  end


In [121]:
module OptionEx = struct
    include Stdlib.Option
    include OptionMonad
    include OptionFunctor
    include OptionApplicative
end

module OptionEx :
  sig
    val none : 'a option
    val some : 'a -> 'a option
    val value : 'a option -> default:'a -> 'a
    val get : 'a option -> 'a
    val join : 'a option option -> 'a option
    val map : ('a -> 'b) -> 'a option -> 'b option
    val fold : none:'a -> some:('b -> 'a) -> 'b option -> 'a
    val iter : ('a -> unit) -> 'a option -> unit
    val is_none : 'a option -> bool
    val is_some : 'a option -> bool
    val equal : ('a -> 'a -> bool) -> 'a option -> 'a option -> bool
    val compare : ('a -> 'a -> int) -> 'a option -> 'a option -> int
    val to_result : none:'e -> 'a option -> ('a, 'e) result
    val to_list : 'a option -> 'a list
    val to_seq : 'a option -> 'a Seq.t
    val return : 'a -> 'a option
    val bind : 'a option -> ('a -> 'b option) -> 'b option
    val ( >>= ) : 'a option -> ('a -> 'b option) -> 'b option
    val ( let* ) : 'a option -> ('a -> 'b option) -> 'b option
    val fmap : ('a -> 'b) -> 'a option -> 'b option
    val ( <!> ) : ('a

In [122]:
open OptionEx;;

let toInt s = int_of_string_opt s;;

let r = 
    toInt "10" >>= fun x -> 
    toInt "20" >>= fun y -> 
    toInt "30" >>= fun z -> return (x + y + z);;

let r =
    let* x = toInt "10" in
    let* y = toInt "20" in
    let* z = toInt "30" in
    return (x + y + z);;

let r = (+) <!> toInt "1" <*> toInt "2";;

let r = 
  let+ x = toInt "10"
  and+ y = toInt "20"
  and+ z = toInt "30" in
  x + y + z;;

val toInt : string -> int option = <fun>


val r : int option = Some 60


val r : int option = Some 60


val r : int OptionEx.t = Some 3


val r : int option = Some 60
